In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os
import torch
import torchvision

# below is the code from the metadata_writer_for_image_classifier.py
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb
from tflite_support.task import vision
from tflite_support.task import core
from tflite_support.task import processor

2022-12-06 19:37:35.218204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 19:37:35.218256: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/joehuang/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# path to a image: tsb/basedata/training/bad-image/2.jpg
img = image.load_img('custom-ava-downloader/Training/training_bad_image/2.jpg')
# save the image to the current directory
img.save('2.jpg')

In [3]:

cv2.imread('2.jpg')

array([[[71, 73, 83],
        [72, 74, 84],
        [74, 76, 86],
        ...,
        [ 5,  7,  8],
        [ 6,  8,  9],
        [ 7,  9, 10]],

       [[77, 79, 89],
        [78, 80, 90],
        [78, 80, 90],
        ...,
        [ 6,  8,  9],
        [ 7,  9, 10],
        [ 8, 10, 11]],

       [[80, 82, 92],
        [80, 82, 92],
        [80, 82, 92],
        ...,
        [ 8, 10, 11],
        [ 8, 10, 11],
        [ 9, 11, 12]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)

In [4]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [5]:
# every pictures have an assigned value
train_dataset = train.flow_from_directory('custom-ava-downloader/Training',
                                            target_size=(200,200),
                                            batch_size= 64,
                                            class_mode='binary')

validation_dataset = train.flow_from_directory('custom-ava-downloader/Validating',
                                            target_size=(200,200),
                                            batch_size=64,
                                            class_mode='binary')
                                            

Found 1189 images belonging to 2 classes.
Found 1165 images belonging to 2 classes.


In [6]:
train_dataset.class_indices # {'bad-image': 0, 'good-image': 1}

{'training_bad_image': 0, 'training_good_image': 1}

In [16]:
""" All of the code below is for the model creation and training. """
# create a model
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200,200,3)), # 16 filters, 3x3 matrix
                                    tf.keras.layers.MaxPooling2D(2,2), # 2x2 matrix
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # 32 filters, 3x3 matrix
                                    tf.keras.layers.MaxPooling2D(2,2), # 2x2 matrix
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), # 64 filters, 3x3 matrix
                                    tf.keras.layers.MaxPooling2D(2,2), # 2x2 matrix
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'), # 512 neurons
                                    tf.keras.layers.Dense(1, activation='sigmoid')]) # 1 neuron

model.summary()

# metadata
model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(lr=0.001),
                metrics=['accuracy'])

# train the model
history = model.fit(train_dataset,
                    steps_per_epoch=8,
                    epochs=14,
                    validation_data=validation_dataset,
                    validation_steps=8,
                    verbose=2)

# save the model
model.save('model.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 99, 99, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 64)      

/home/joehuang/.local/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/14
8/8 - 28s - loss: 2.0008 - accuracy: 0.5820 - val_loss: 0.6497 - val_accuracy: 0.6504 - 28s/epoch - 4s/step
Epoch 2/14
8/8 - 26s - loss: 0.6380 - accuracy: 0.6825 - val_loss: 0.6404 - val_accuracy: 0.6660 - 26s/epoch - 3s/step
Epoch 3/14
8/8 - 25s - loss: 0.6260 - accuracy: 0.6777 - val_loss: 0.6538 - val_accuracy: 0.6426 - 25s/epoch - 3s/step
Epoch 4/14
8/8 - 24s - loss: 0.6099 - accuracy: 0.6758 - val_loss: 0.6319 - val_accuracy: 0.6934 - 24s/epoch - 3s/step
Epoch 5/14
8/8 - 26s - loss: 0.6514 - accuracy: 0.6094 - val_loss: 0.6550 - val_accuracy: 0.6543 - 26s/epoch - 3s/step
Epoch 6/14
8/8 - 27s - loss: 0.6016 - accuracy: 0.7093 - val_loss: 0.6744 - val_accuracy: 0.6426 - 27s/epoch - 3s/step
Epoch 7/14
8/8 - 23s - loss: 0.7513 - accuracy: 0.7320 - val_loss: 0.6682 - val_accuracy: 0.6641 - 23s/epoch - 3s/step
Epoch 8/14
8/8 - 25s - loss: 0.6242 - accuracy: 0.7361 - val_loss: 0.6883 - val_accuracy: 0.5957 - 25s/epoch - 3s/step
Epoch 9/14
8/8 - 31s - loss: 0.5562 - accuracy: 

In [17]:
""" All the code below is for the metadata """
# convert the model to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save the tflite model
open('model.tflite', 'wb').write(tflite_model)

# create a label file for the tflite model
labels = '\n'.join(sorted(train_dataset.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)


# metadata for the tflite model
model_file = 'model.tflite'
label_file = 'labels.txt'
model_type = 'image.classification'
name = 'Image Classification'
description = 'Identify if the image is good or bad'
version = 'v1'
image_width = 200
image_height = 200
image_min = 0
image_max = 255
mean = [0]
std = [255]
is_quantized = False
model_author = 'MVP'
model_license = 'Apache License. Version 2.0 http://www.apache.org/licenses/LICENSE-2.0.'

# create the metadata
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = name
model_meta.description = description
model_meta.version = version
model_meta.author = model_author
model_meta.license = model_license

# # create the associated file
# associated

# create the input image
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = 'input_image'
input_meta.description = ('Input image to be classified.')
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = mean
input_normalization.options.std = std
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [image_max]
input_stats.min = [image_min]
input_meta.stats = input_stats

# create the output
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = 'probability'
output_meta.description = ('Probabilities of the label respectively.')
output_meta.content = _metadata_fb.ContentT()
output_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_meta.stats = _metadata_fb.StatsT()
output_meta.stats.max = [1.0]
output_meta.stats.min = [0.0]

# create the subgraph
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
subgraph.inputTensorNames = ['input_1']
subgraph.outputTensorNames = ['dense_1']
model_meta.subgraphMetadata = [subgraph]

# create the buffer
b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

# create the populator
populator = _metadata.MetadataPopulator.with_model_file(model_file)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([label_file])
populator.populate()



INFO:tensorflow:Assets written to: /tmp/tmpmdm8qnko/assets


INFO:tensorflow:Assets written to: /tmp/tmpmdm8qnko/assets
2022-12-06 19:51:23.722306: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-12-06 19:51:23.722349: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-12-06 19:51:23.722863: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpmdm8qnko
2022-12-06 19:51:23.726162: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-12-06 19:51:23.726232: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpmdm8qnko
2022-12-06 19:51:23.738192: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-12-06 19:51:23.835240: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmpmdm8qnko
2022-12-06 19:51:23.857078: I tensorflow/cc/saved_model/loader.cc:283] SavedMode

In [18]:
"""Save the metadata to a file. JSON"""
# visualize the metadata
displayer = _metadata.MetadataDisplayer.with_model_file(model_file)
export_json_file = 'metadata.json'
json_file = displayer.get_metadata_json()
with open(export_json_file, 'w') as f:
    f.write(json_file)
print('Metadata json saved to: ', export_json_file)

Metadata json saved to:  metadata.json


In [10]:
"""Everything above should be what you need to create a tflite model with metadata. Everything below is old code which is not needed."""

'Everything above should be what you need to create a tflite model with metadata. Everything below is old code which is not needed.'

In [11]:
"""Inludeed in the above code already"""
# # compile the model
# model.compile(loss='binary_crossentropy',
#                 optimizer=RMSprop(lr=0.001),
#                 metrics=['accuracy'])

'Inludeed in the above code already'

In [12]:
"""Inludeed in the above code already"""
# # train the model
# history = model.fit(train_dataset,
#                     steps_per_epoch=19, 
#                     epochs=10,
#                     validation_data=validation_dataset)

"""Don't need this

    This is just to visualize the accuracy and loss of the model"""
# # plot the accuracy and loss into a graph
# acc = history.history['accuracy'] # accuracy
# val_acc = history.history['val_accuracy'] # validation accuracy
# loss = history.history['loss'] # loss
# val_loss = history.history['val_loss'] # validation loss

# # note: to increase the accuracy, we can 
# # 1. get more data
# # 2. add more layers 
# # 3. change image size, not too small or too large
# # 4. increase the number of epochs

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend(loc=0)
# plt.figure()

# plt.show()

"Don't need this\n\n    This is just to visualize the accuracy and loss of the model"

In [13]:
"""Don't need this

    This is just to visualize the accuracy and loss of the model"""
# plt.plot(epochs, loss, 'r', label='Training Loss')
# plt.plot(epochs, val_loss, 'b', label='Validation Loss')
# plt.title('Training and validation loss')
# plt.legend(loc=0)
# plt.figure()

# plt.show()

"Don't need this\n\n    This is just to visualize the accuracy and loss of the model"

In [14]:
"""Don't need this

    uncomment the whole thing if you want to see some of the output of the test dataset"""
# # predict the image in the custom-ava-downloader/validation_image folder with the model
# # print the prediction result
# # if the prediction result is 1, then the image is a good image
# # if the prediction result is 0, then the image is a bad image
# import numpy as np
# import json
# from keras.preprocessing import image

# # read JSON file, for comparsion later on with prediction result
# with open('/mnt/c/Users/joe19/Desktop/EECS441/custom-ava-downloader/image_data.json') as json_file:
#     data = json.load(json_file)

# # loop through the images in the custom-ava-downloader/validation_image folder and predict the image
# for image_name in os.listdir('custom-ava-downloader/Testing'):

#     # resize this image 
#     test_image = image.load_img('custom-ava-downloader/Testing/' + image_name, target_size = (200, 200))

#     # print the image 
#     plt.imshow(test_image)
#     plt.show()

#     # convert the image to an array
#     test_image = image.img_to_array(test_image)

#     # expand the dimension of the image
#     test_image = np.expand_dims(test_image, axis = 0)

#     # predict the image
#     result = model.predict(test_image)
#     if result[0][0] == 1:
#         prediction = 'good image'
#     else:
#         prediction = 'bad image'
#     print(image_name + " is a " + prediction + " with a prediction result of " + str(result[0][0]))

"Don't need this\n\n    uncomment the whole thing if you want to see some of the output of the test dataset"

In [15]:
# # convert the model to a tflite model
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # save the tflite model
# open("converted_model.tflite", "wb").write(tflite_model)